<a href="https://colab.research.google.com/github/arutraj/.githubcl/blob/main/%5BSolution%5D_Assignment_Big_Data_%26_Spark_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install pyspark and customize Colab configuration


The pyspark installation will persist until the runtime is recycled.

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
!pip install ucimlrepo

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,665 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,105 kB]
Ign:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy Rele

'/usr/local/lib/python3.10/dist-packages/pyspark'

# Initialize Spark

In [2]:
import pyspark
from pyspark import SparkContext

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

sc = spark.sparkContext
spark

# Load Bank Dataset

In [14]:
import pandas as pd
from ucimlrepo import fetch_ucirepo

# fetch dataset
bank_marketing = fetch_ucirepo(id=222)

# data (as pandas dataframes)
dataset_X = bank_marketing.data.features
dataset_y = bank_marketing.data.targets

dataset = pd.concat([dataset_X, dataset_y], axis=1)
dataset.rename(columns={"y": "deposit"}, inplace=True)

sdf = spark.createDataFrame(dataset)
sdf.createOrReplaceTempView("spktest_bank")

# Questions

In [15]:
sdf.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+-----------+-----+--------+--------+-----+--------+--------+-------+
|age|         job| marital|education|default|balance|housing|loan|contact|day_of_week|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+------------+--------+---------+-------+-------+-------+----+-------+-----------+-----+--------+--------+-----+--------+--------+-------+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|    NaN|          5|  may|     261|       1|   -1|       0|     NaN|     no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|    NaN|          5|  may|     151|       1|   -1|       0|     NaN|     no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|    NaN|          5|  may|      76|       1|   -1|       0|     NaN|     no|
| 47| blue-collar| married|      NaN|     no|   1506|    yes|  no|    NaN|          5|  may|      92|       1|   -1|       0|     NaN|     no|

In [17]:
spark.sql("SELECT * FROM spktest_bank").show()

+---+------------+--------+---------+-------+-------+-------+----+-------+-----------+-----+--------+--------+-----+--------+--------+-------+
|age|         job| marital|education|default|balance|housing|loan|contact|day_of_week|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+------------+--------+---------+-------+-------+-------+----+-------+-----------+-----+--------+--------+-----+--------+--------+-------+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|    NaN|          5|  may|     261|       1|   -1|       0|     NaN|     no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|    NaN|          5|  may|     151|       1|   -1|       0|     NaN|     no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|    NaN|          5|  may|      76|       1|   -1|       0|     NaN|     no|
| 47| blue-collar| married|      NaN|     no|   1506|    yes|  no|    NaN|          5|  may|      92|       1|   -1|       0|     NaN|     no|

1. Using Spark SQL, find marital status and corresponding total count for all the people.

In [19]:
# Solution
spark.sql("SELECT marital, COUNT(*) AS num_people FROM spktest_bank GROUP BY marital").show()

+--------+----------+
| marital|num_people|
+--------+----------+
|divorced|      5207|
| married|     27214|
|  single|     12790|
+--------+----------+



2. Find number of people with balance greater than or equal to 80<sup>th</sup> percentile.

In [22]:
# Solution
spark.sql("SELECT COUNT(*) AS num_people FROM spktest_bank WHERE balance < (SELECT PERCENTILE(balance, 0.75) FROM spktest_bank)").show()

+----------+
|num_people|
+----------+
|     33907|
+----------+



3. Find the Average Balance of clients who made a deposit.

In [25]:
# Solution
spark.sql("SELECT AVG(balance) AS avg_balance FROM spktest_bank GROUP BY deposit HAVING deposit='yes'").show()

+------------------+
|       avg_balance|
+------------------+
|1804.2679145396105|
+------------------+



4. Write a Spark SQL query using Common Table Expressions (CTEs) and LIMIT clause to find the job category with the highest average balance among clients who have not defaulted on their loans and have made a deposit. The query should consider only clients aged between 30 and 50.

In [27]:
# Solution
spark.sql("""
  WITH clients AS (
    SELECT * FROM spktest_bank WHERE default='no' and deposit='yes' and age BETWEEN 30 AND 50
  )

  SELECT job, AVG(balance) AS avg_balance FROM clients GROUP BY job ORDER BY avg_balance DESC LIMIT 3
  """).show()

+----------+------------------+
|       job|       avg_balance|
+----------+------------------+
|   retired|3030.1666666666665|
|management|2099.8103837471785|
| housemaid|           2034.82|
+----------+------------------+



5. Write a Spark SQL query using CTE and WINDOW function to find the job category with the highest average balance among clients who have not defaulted on their loans and have made a deposit. The query should consider only clients aged between 30 and 50.

In [30]:
# Solution
spark.sql("""
  WITH clients AS (
    SELECT * FROM spktest_bank WHERE default='no' and deposit='yes' and age BETWEEN 30 AND 50
  ),

  avg_bal AS (
    SELECT job, AVG(balance) AS avg_balance FROM clients GROUP BY job
  )

  SELECT job, avg_balance, rank
  FROM
    (SELECT *, ROW_NUMBER() OVER (ORDER BY avg_balance DESC) AS rank FROM avg_bal)
  WHERE rank < 6
  """).show()

+-------------+------------------+----+
|          job|       avg_balance|rank|
+-------------+------------------+----+
|      retired|3030.1666666666665|   1|
|   management|2099.8103837471785|   2|
|    housemaid|           2034.82|   3|
|   technician| 1923.380357142857|   4|
|self-employed|         1805.6875|   5|
+-------------+------------------+----+



6. Write a Spark SQL query to find the month with the highest number of clients who have made a deposit and have not defaulted on their loans.

In [32]:
# Solution
spark.sql("SELECT month, COUNT(*) AS num_clients FROM spktest_bank WHERE default='no' AND deposit='yes' GROUP BY month ORDER BY num_clients DESC LIMIT 2").show()

+-----+-----------+
|month|num_clients|
+-----+-----------+
|  may|        917|
|  aug|        683|
+-----+-----------+



7. Write a Spark SQL query using an User Defined Function(UDF) to calculate average duration in weeks for each job type?

In [35]:
# Solution
def get_duration_in_weeks(days):
  return days/7

spark.udf.register("get_duration_in_weeks", get_duration_in_weeks, T.DoubleType())

spark.sql("SELECT job, get_duration_in_weeks(AVG(duration)) AS avg_duration_in_weeks FROM spktest_bank GROUP BY job order by avg_duration_in_weeks desc").show()

+-------------+---------------------+
|          job|avg_duration_in_weeks|
+-------------+---------------------+
|   unemployed|    41.22048021050323|
|      retired|    41.05161534578496|
|self-employed|     38.3081516330408|
|  blue-collar|    37.55736597968411|
|     services|    37.04553270513791|
| entrepreneur|    36.61562109712749|
|   management|   36.285110110866086|
|   technician|    36.12928035502736|
|       admin.|    35.27096168190734|
|      student|    35.23667377398721|
|    housemaid|    35.11785714285714|
|          NaN|    33.94444444444444|
+-------------+---------------------+



8. Using CTE and JOIN operator, write a Spark SQL query to find the Median Duration and Median Balance for each job type.

In [36]:
# Solution
spark.sql("""
  WITH median_duration_cte AS (
    SELECT job, PERCENTILE(duration, 0.5) AS median_duration FROM spktest_bank GROUP BY job
  ),

  median_balance_cte AS (
    SELECT job, PERCENTILE(balance, 0.5) AS median_balance FROM bank GROUP BY job
  )

  SELECT
    median_duration_cte.job,
    median_duration,
    median_balance
  FROM
    median_duration_cte
  JOIN
    median_balance_cte
  ON median_duration_cte.job = median_balance_cte.job
""").show()

+-------------+---------------+--------------+
|          job|median_duration|median_balance|
+-------------+---------------+--------------+
|   management|          173.0|         572.0|
|      retired|          204.0|         787.0|
|self-employed|          179.0|         526.0|
|      student|          180.0|         502.0|
|  blue-collar|          186.0|         388.0|
| entrepreneur|          178.0|         352.0|
|       admin.|          174.0|         396.0|
|   technician|          176.0|         421.0|
|          NaN|          165.0|         677.0|
|     services|          186.0|         339.5|
|    housemaid|          163.0|         406.0|
|   unemployed|          200.0|         529.0|
+-------------+---------------+--------------+



9. Using Statistic package from MLlib, find the Pearson correlation between Balance and Duration.

In [39]:
# Solution
from pyspark.mllib.stat import Statistics

df_x = spark.sql("SELECT balance, duration FROM spktest_bank").rdd.map(lambda x: x[0])
df_y = spark.sql("SELECT duration FROM spktest_bank").rdd.map(lambda x: x[0])

print(df_x, df_y)
print("Pearson Correlation: ", Statistics.corr(df_x, df_y, method='pearson'))

PythonRDD[249] at RDD at PythonRDD.scala:53 PythonRDD[250] at RDD at PythonRDD.scala:53
Pearson Correlation:  0.02156038049466903


10. Build a k-means model with k=2 clusters on a balanced dataset and evaluate the clusters using Silhouette coefficient.

In [43]:
# Solution

# Stratified sampling to balance the dataset
fractions = {
    "no": 0.10,
    "yes": 1.0
}

sdf = sdf.sampleBy("deposit", fractions, seed=212)
sdf = sdf.select(
    'age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan',
    'contact', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'deposit'
)
cols = sdf.columns
sdf.printSchema()

root
 |-- age: long (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: long (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- campaign: long (nullable = true)
 |-- pdays: long (nullable = true)
 |-- previous: long (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [44]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

stages = []
categoricalColumns = [
    'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome'
]

for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(
        inputCols=[stringIndexer.getOutputCol()],
        outputCols=[categoricalCol + "classVec"]
    )
    stages += [stringIndexer, encoder]

label_stringIdx = StringIndexer(inputCol = 'deposit', outputCol = 'label')
stages += [label_stringIdx]
numericCols = ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [45]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(sdf)
sdf = pipelineModel.transform(sdf)
selectedCols = ['label', 'features'] + cols
sdf = sdf.select(selectedCols)
sdf.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- age: long (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: long (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- campaign: long (nullable = true)
 |-- pdays: long (nullable = true)
 |-- previous: long (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [46]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Trains a k-means model.
kmeans = KMeans(featuresCol = 'features', k=2, seed=1)
model = kmeans.fit(sdf)

# Make predictions
predictions = model.transform(sdf)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.9760083517189627




---

